<a href="https://colab.research.google.com/github/waltz2u/bd/blob/master/Basic_Spark_RDD_MorningStar_Rating_and_Stocks_EMR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Spark RDD with MorningStar Rating and Stock data

In [0]:
sc.list_packages()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1584068654270_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version
-------------------------- -------
beautifulsoup4             4.8.1  
boto                       2.49.0 
jmespath                   0.9.4  
lxml                       4.4.2  
mysqlclient                1.4.6  
nltk                       3.4.5  
nose                       1.3.4  
numpy                      1.14.5 
pip                        20.0.2 
py-dateutil                2.2    
python36-sagemaker-pyspark 1.2.6  
pytz                       2019.3 
PyYAML                     3.11   
setuptools                 46.0.0 
six                        1.13.0 
soupsieve                  1.9.5  
wheel                      0.34.2 
windmill                   1.6

## MorningStar Ratings 

First read data in

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate() #

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [0]:
lines = spark.sparkContext.textFile('s3://uconn19/morningstar.csv')
lines.take(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ProFunds Short Precious Metals Inv,SPPIX,NA', 'ProFunds Short Precious Metals Svc,SPPSX,NA', 'ProFunds Biotechnology UltraSector Inv,BIPIX,4']

In [0]:
ratings = lines.map(lambda x: x.split(',')[2])
ratings.take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['NA', 'NA', '4', '3', '5', '5', '4', '4', '4', '4']

In [0]:
result = ratings.countByValue()
print(result)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defaultdict(<class 'int'>, {'NA': 19, '4': 19, '3': 19, '5': 18, '1': 24})

In [0]:
import collections
sortedCounts = collections.OrderedDict(sorted(result.items()))
for key, value in sortedCounts.items():
    print ("%s %i" % (key, value))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1 24
3 19
4 19
5 18
NA 19

## Mean Price by Month

Given date and stock price, returns average price by month

In [0]:
# this function gets month, year and adjusted closed price
def parseLine(line):
    fields = line.split(',')
    datetimedata = fields[0]
    datetimearray = datetimedata.split('/')
    monthyear = datetimearray[0]+datetimearray[2]
    adjclose = float(fields[6])
    return (monthyear, adjclose)

lines = spark.sparkContext.textFile("s3://uconn19/adjclose.csv")

# lines is an RDD, take some to check
lines.take(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['1/28/2015,49.799999,50.32,46.299999,46.459999,84839700,46.459999', '1/27/2015,49.139999,49.279999,47.66,47.990002,45777200,47.990002', '1/26/2015,49.57,49.790001,49.07,49.439999,18976400,49.439999']

In [0]:
# gets month, year and adjusted closed price
rdd = lines.map(parseLine)
rdd.take(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('12015', 46.459999), ('12015', 47.990002), ('12015', 49.439999)]

In [0]:
# mapValues to gets a tuple of (adjusted price, 1) for each month
rdd.mapValues(lambda x: (x, 1)).take(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('12015', (46.459999, 1)), ('12015', (47.990002, 1)), ('12015', (49.439999, 1))]

In [0]:
# then add tuples together using reduceByKey
priceByMonthYear = rdd.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
priceByMonthYear.take(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('12015', (871.899997, 18)), ('112014', (952.880004, 19)), ('92014', (854.119997, 21))]

In [0]:
# then finally mapValues to calculate average price 
averagesByMonthYear = priceByMonthYear.mapValues(lambda x: x[0] / x[1])
results = averagesByMonthYear.collect()
for result in results:
    print (result)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('12015', 48.438888722222224)
('112014', 50.15157915789474)
('92014', 40.67238080952381)
('82014', 36.87619052380952)
('72014', 35.097272409090905)
('32014', 37.71476223809524)
('22014', 37.43578963157895)
('82013', 27.525454454545457)
('72013', 27.304545590909093)
('62013', 25.911499999999997)
('52013', 26.159545227272723)
('12013', 19.815714523809525)
('102012', 16.12428576190476)
('82012', 15.226086956521744)
('72012', 15.77476195238095)
('52012', 15.358181818181817)
('42012', 15.253499999999997)
('22012', 15.371500000000003)
('122011', 15.678571333333334)
('72011', 14.505)
('62011', 15.178181818181816)
('52011', 16.98523780952381)
('32011', 16.660869608695652)
('12011', 16.42199995)
('112010', 16.419523761904763)
('82010', 13.796363636363639)
('72010', 14.474761904761904)
('52010', 15.935499949999999)
('32010', 16.303913260869564)
('22010', 15.268947368421053)
('102009', 16.94318168181818)
('92009', 16.150952285714283)
('82009', 14.75095238095238)
('122014', 50.34045431818181)
('10